In [13]:
import io
import boto3
import pandas as pd

def read_datasets():
    # Read three datasets stored in the Amazon S3 bucket
   # bucket = "faostat-ml"
    file_name = "Emissions_Totals_E_All_Data_(Normalized).csv"
    #s3_client = boto3.client("s3")
    #obj = s3_client.get_object(Bucket=bucket, Key=file_name)
    df_emission = pd.read_csv("Emissions_Totals_E_All_Data_(Normalized).csv",encoding='latin-1')
    df_emission.drop(df_emission.columns[df_emission.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

    #fix the feature names
    df_emission = df_emission.rename(columns={'Item': 'EmissionItem','Value': 'EmissionValue','Element': 'EmissionElement','Unit': 'EmissionUnit'})
    print(df_emission.head(5))

    file_name = "Production_Crops_Livestock_E_All_Data_(Normalized).csv"
    #obj = s3_client.get_object(Bucket=bucket, Key=file_name)
    df_prod = pd.read_csv("Production_Crops_Livestock_E_All_Data_(Normalized).csv",encoding='latin-1')
    df_prod.drop(df_prod.columns[df_prod.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
    df_prod.drop('Unit', axis=1, inplace=True)
    print(df_prod.head(5))


    file_name = "Forestry_E_All_Data_(Normalized).csv"
    #obj = s3_client.get_object(Bucket=bucket, Key=file_name)
    df_forest = pd.read_csv("Forestry_E_All_Data_(Normalized).csv",encoding='latin-1')
    df_forest.drop(df_forest.columns[df_forest.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
    print(df_forest.head(5))
    return df_emission, df_prod, df_forest

In [14]:
def pre_processing():
    remove_rows_df = pre_process_df.copy()
    
    # Removes area, source and emission unit that place no significant role
    remove_rows_df.drop('Area', axis=1, inplace=True)
    remove_rows_df.drop('Source', axis=1, inplace=True)
    remove_rows_df.drop('EmissionUnit', axis=1, inplace=True)
    
    # Remove redudant instance
    remove_rows_df = remove_rows_df[remove_rows_df.Element != "Area harvested"]
    remove_rows_df = remove_rows_df[remove_rows_df.Element != "Production"]
    remove_rows_df = remove_rows_df[remove_rows_df.Element != "Producing Animals/Slaughtered"]
    remove_rows_df = remove_rows_df[remove_rows_df.EmissionElement != "Indirect emissions (N2O)"]
    remove_rows_df = remove_rows_df[remove_rows_df.EmissionElement != "Direct emissions (N2O)"]
    remove_rows_df.index = remove_rows_df.Year
    remove_rows_df.drop('Year', axis=1, inplace=True)

    #identify partial string to look for
    discard = ['from']
    remove_rows_df = remove_rows_df[~remove_rows_df.EmissionElement.str.contains('|'.join(discard))]

    # Combine two features
    remove_rows_df["Emission"] = remove_rows_df["EmissionItem"] + str("_") + remove_rows_df["EmissionElement"]
    remove_rows_df.drop('EmissionItem', axis=1, inplace=True)
    remove_rows_df.drop('EmissionElement', axis=1, inplace=True)

    # Remove redudant instance
    remove_rows_df = remove_rows_df[remove_rows_df.Element != 'Import Value']
    remove_rows_df = remove_rows_df[remove_rows_df.Element != 'Export Value']
    
    emission_list = list(pre_process_df.EmissionItem.unique())

    # Create pivot table for production items + forestry products based on year and item
    df_item = remove_rows_df.pivot_table(index=['Year'], 
                columns=['Item'], values='Value')
    
    # Remove columns that have atleast one NaN value since it would affect the forecast
    sum = df_item.isnull().sum(axis = 0)
    for items in sum.iteritems():
        if(items[1]>0):
            df_item.drop(items[0], axis=1, inplace=True)
    nan_cols = [i for i in df_item.columns if df_item[i].isnull().any()]

    # Create pivot table for emissions
    df_emi = remove_rows_df.pivot_table(index=['Year'], 
                columns=['Emission'], values='EmissionValue')
    # Remove columns that have atleast one NaN value since it would affect the forecast
    sum = df_emi.isnull().sum(axis = 0)
    for items in sum.iteritems():
        if(items[1]>0):
            df_emi.drop(items[0], axis=1, inplace=True)
    nan_cols = [i for i in df_emi.columns if df_emi[i].isnull().any()]
    
    display(df_item.head(5))
    display(df_emi.head(5))
    return df_item, df_emi, emission_list

In [15]:
# Perform granger causation test - to check the influence of one variable over another

def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
    import numpy as np
    from statsmodels.tsa.stattools import grangercausalitytests
    maxlag=12
    """Check Granger Causality of all possible combinations of the Time series.
    The rows are the response variable, columns are predictors. The values in the table 
    are the P-Values. P-Values lesser than the significance level (0.05), implies 
    the Null Hypothesis that the coefficients of the corresponding past values is 
    zero, that is, the X does not cause Y can be rejected.
    """
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

In [16]:
# Perform ADF test to check if each series is stationary or not

def adfuller_test(series, signif=0.05, name='', verbose=False):
    i = 0
    from statsmodels.tsa.api import VECM
    import semopy
    from statsmodels.tsa.stattools import adfuller
    from statsmodels.tools.eval_measures import rmse, aic
    non_stationary = []
    """Perform ADFuller to test for Stationarity of given series and print report"""
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue'] 
    def adjust(val, length= 6): return str(val).ljust(length)

    # Print Summary
    print(f'    Augmented Dickey-Fuller Test on "{name}"', "\n   ", '-'*47)
    print(f' Null Hypothesis: Data has unit root. Non-Stationary.')
    print(f' Significance Level    = {signif}')
    print(f' Test Statistic        = {output["test_statistic"]}')
    print(f' No. Lags Chosen       = {output["n_lags"]}')
    
    for key,val in r[4].items():
        i = i + 1
        print(f' Critical value {adjust(key)} = {round(val, 3)}')

    if p_value <= signif:
        print(f" => P-Value = {p_value}. Rejecting Null Hypothesis.")
        print(f" => Series is Stationary.")
    else:
        print(f" => P-Value = {p_value}. Weak evidence to reject the Null Hypothesis.")
        print(f" => Series is Non-Stationary.")

In [17]:
# Revert back the differencing to get the forecast to original scale

def invert_transformation(df_train, df_forecast, second_diff=False):
    df_fc = df_forecast.copy()
    columns = df_train.columns
    for col in columns:        
        # Roll back 2nd Diff
        if second_diff:
            df_fc[str(col)+'_1d'] = (df_train[col].iloc[-1]-df_train[col].iloc[-2]) + df_fc[str(col)+'_2d'].cumsum()
        # Roll back 1st Diff
        df_fc[str(col)+'_forecast'] = df_train[col].iloc[-1] + df_fc[str(col)+'_1d'].cumsum()
    return df_fc

In [18]:
# Perform stationary test and differencing


""" Note: Comment out the 'for loop' when adf fuller test results need to be displayed"""

def stationary(df):
    nobs = 9
    df_train, df_test = df[0:-nobs], df[-nobs:]
    
    # ADF Test on each column
    #for name, column in df_train.iteritems():
        #adfuller_test(column, name=column.name)
        #print('\n')
        
    # 1st difference
    df_differenced = df_train.diff().dropna()
    #for name, column in df_differenced.iteritems():
        #adfuller_test(column, name=column.name)
        #print('\n')
        
    # Second Differencing
    df_differenced = df_differenced.diff().dropna()   
    #for name, column in df_differenced.iteritems():
        #adfuller_test(column, name=column.name)
        #print('\n')
        
    return df_differenced, df_train, df_test

In [19]:
# The function that performs forecasting

mae =[]
mse = []
sqr = []
    
def forecasting(df_item, df_emi, emission_list):
    import warnings
    import pandas as pd
    from numpy import sqrt 
    import plotly.graph_objs as go
    from statsmodels.tsa.api import VAR
    from sklearn.metrics import mean_absolute_error, mean_squared_error
    warnings.simplefilter(action='ignore', category=FutureWarning)
    col = df_item.columns
    pandas_df = pd.DataFrame()
    nobs = 9

    # iterate through every item - production and forestry product - for the country/area specified
    for i in col:
        selected_columns = df_item[i]

        new_df = selected_columns.copy()
        new_merged_df = pd.merge(new_df, df_emi, on=['Year'])

        for j in emission_list:
            emission = new_merged_df.filter(regex=j)
            col = list(emission.columns) 
            new_merged_df[j] = new_merged_df[col].sum(axis=1)

        pandas_df = new_merged_df[new_merged_df.columns.intersection(emission_list)]
        pandas_df[str(new_merged_df.columns[0])] = new_merged_df[str(new_merged_df.columns[0])]
        pandas_df = pandas_df.loc[:, (pandas_df != 0).any(axis=0)]
         
        """ Note: comment out if granger test needs to be performed"""
        #print("Granger's causality test:")
        #display(grangers_causation_matrix(pandas_df, variables = pandas_df.columns))        
        
        historical_split1 = pandas_df.iloc[:50,:]
        historical_split2 = pandas_df.iloc[50:,:]
    
        df_differenced,df_train,_ = stationary(historical_split1)
        model = VAR(df_differenced)
        model_fitted = model.fit(6)

        # Get the lag order
        lag_order = model_fitted.k_ar

        # Input data for forecasting
        forecast_input = df_differenced.values[-lag_order:]
        fc = model_fitted.forecast(y=forecast_input, steps=nobs)
        df_forecast = pd.DataFrame(fc, index=historical_split1.index[-nobs:], columns=historical_split1.columns + '_2d')

        df_results = invert_transformation(df_train, df_forecast, second_diff=True)
        d = historical_split1.tail(nobs)
        d.reset_index(inplace = True)


        range_year = pd.date_range(start = str(d.Year.iloc[-1]), periods = (len(d)+1), freq = 'A')
        range_year = range_year.year
        year = pd.DataFrame({'Year': range_year})
        d = d.append(year)
        year_forecast = d['Year'].iat[-1]

        d.set_index('Year', inplace = True)
        d = d.tail(nobs)
            
        df_results.index = d.index
        mean_old = historical_split1[i].mean()
        recent_forecast = df_results[i+"_forecast"]
        recent_forecast = recent_forecast.iat[-1]

        #combining predicted and real data set
        combine = pd.concat([df_results[i+"_forecast"], historical_split2[i]], axis=1)
        combine = combine.round(decimals=2)
        combine = combine.reset_index()

        combine[i+"_Unscaled"] = combine[i]
        combine[i+ "_Forecast_Unscaled"] = combine[i+"_forecast"]

        combine[i]=(combine[i]-combine[i].min())/(combine[i].max()-combine[i].min())
        combine[i+"_forecast"]=(combine[i+"_forecast"]-combine[i+"_forecast"].min())/(combine[i+"_forecast"].max()-combine[i+"_forecast"].min())


        display(combine)
        null_check = combine.isnull().sum().sum()
        if null_check > 0:
            continue
        #Forecast metrics

        mae.append(mean_absolute_error(combine[i].values, combine[i+"_forecast"].values))
        mse.append(mean_squared_error(combine[i].values, combine[i+"_forecast"].values))
        sqr.append(sqrt(mean_squared_error(combine[i].values, combine[i+"_forecast"].values)))
        
        fig = go.Figure()
        n = df_results.index[0]
        fig.add_trace(go.Scatter(x = pandas_df.index[-200:], y = pandas_df[str(i)][-200:], marker = dict(color ="red"), name = "Actual close price"))
        fig.add_trace(go.Scatter(x = df_results.index, y = df_results[str(i)+'_forecast'], marker=dict(color = "green"), name = "Future prediction"))
        fig.update_xaxes(showline = True, linewidth = 2, linecolor='black', mirror = True, showspikes = True,)
        fig.update_yaxes(showline = True, linewidth = 2, linecolor='black', mirror = True, showspikes = True,)
        fig.update_layout(title= "9 Years Forecast", yaxis_title = str(i), hovermode = "x", hoverdistance = 100) #, # Distance to show hover label of data point spikedistance = 1000,shapes = [dict( x0 = n, x1 = n, y0 = 0, y1 = 1, xref = 'x', yref = 'paper', line_width = 2)], annotations = [dict(x = n, y = 0.05, xref = 'x', yref = 'paper', showarrow = False, xanchor = 'left', text = 'Prediction')])
        fig.update_layout(autosize = False, width = 1000, height = 400,)
        fig.show()
        
        
        
        


In [20]:
def forecasting1(df_item, df_emi, emission_list):
    import warnings
    import pandas as pd
    from numpy import sqrt 
    import plotly.graph_objs as go
    from statsmodels.tsa.api import VAR
    from sklearn.metrics import mean_absolute_error, mean_squared_error
    from statsmodels.stats.stattools import durbin_watson

    warnings.simplefilter(action='ignore', category=FutureWarning)
    col = df_item.columns
    pandas_df = pd.DataFrame()
    nobs = 9

    # iterate through every item - production and forestry product - for the country/area specified
    for i in col:
        selected_columns = df_item[i]

        new_df = selected_columns.copy()
        new_merged_df = pd.merge(new_df, df_emi, on=['Year'])

        for j in emission_list:
            emission = new_merged_df.filter(regex=j)
            col = list(emission.columns) 
            new_merged_df[j] = new_merged_df[col].sum(axis=1)

        pandas_df = new_merged_df[new_merged_df.columns.intersection(emission_list)]
        pandas_df[str(new_merged_df.columns[0])] = new_merged_df[str(new_merged_df.columns[0])]
        pandas_df = pandas_df.loc[:, (pandas_df != 0).any(axis=0)]
         
        """ Note: comment out if granger test needs to be performed"""
        #print("Granger's causality test:")
        #display(grangers_causation_matrix(pandas_df, variables = pandas_df.columns))        
        
        historical_split1 = pandas_df.iloc[:50,:]
        historical_split2 = pandas_df.iloc[50:,:]
    
        df_differenced,df_train,_ = stationary(historical_split1)
        model = VAR(df_differenced)
        model_fitted = model.fit(6)
        
        results = model.fit(maxlags=2, ic='aic')

# make predictions for the next 10 periods
        lag_order = results.k_ar
        predictions = results.forecast(df_differenced.values[-lag_order:], steps=10)

        # calculate the residuals
        residuals = df_differenced.values[-10:] - predictions

        # plot the residuals
        plt.plot(residuals)
        plt.title('Residuals')
        plt.show()

        # calculate the Durbin-Watson statistic for each variable
        for i, col in enumerate(df_differenced.columns):
            dw = durbin_watson(residuals[:, i])
            print(f'Durbin-Watson statistic for {col}: {dw}')
            
            


In [21]:
def time_series_prediction():
    pre_process_df = pd.DataFrame()
    combined_df = pd.DataFrame()

    import numpy as np
    import seaborn as sb
    import ipywidgets as widgets
    import matplotlib.pyplot as plt
    from sklearn import preprocessing
    from sklearn.preprocessing import LabelEncoder
    from ipywidgets import Layout, Button, Box, FloatText, Textarea, Dropdown, Label, IntSlider

    def unique_sorted_values_plus_ALL(array):
        unique = array.unique().tolist()
        unique.sort()
        return unique
    output_area = widgets.Output()
    print("Area:")
    dropdown_area = widgets.Dropdown(options=unique_sorted_values_plus_ALL(df_prod.Area))
    def dropdown_area_eventhandler(change):
        output_area.clear_output()
        with output_area:
            if(change.new):
                country_df_prod = df_prod[df_prod.Area == change.new]
                country_df_forest = df_forest[df_forest.Area == change.new]
                country_df_emission = df_emission[df_emission.Area == change.new]

                prod_forest_df = pd.concat([country_df_prod, country_df_forest], ignore_index=True)
                global pre_process_df, combined_df
                pre_process_df = pd.merge(prod_forest_df, country_df_emission, on=['Year','Area'])
    dropdown_area.observe(dropdown_area_eventhandler, names='value')
    display(dropdown_area)

In [22]:
# Run the functions from this module onwards for time series prediction

df_emission, df_prod, df_forest = read_datasets()

FileNotFoundError: [Errno 2] No such file or directory: 'Emissions_Totals_E_All_Data_(Normalized).csv'

In [23]:
time_series_prediction()

Area:


NameError: name 'df_prod' is not defined

The below cell outputs consist of Granger causality matrix, ADF fuller test results for each feature and prediction graphs

In [11]:
import warnings
warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')

display(pre_process_df.head(5))
df_item, df_emi, emission_list = pre_processing()
mae,mse,sqr = forecasting(df_item, df_emi, emission_list)

NameError: name 'pre_process_df' is not defined

In [12]:
import matplotlib.pyplot as plt
import statsmodels.tsa.api as smt
import seaborn as sns
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')

display(pre_process_df.head(5))
df_item, df_emi, emission_list = pre_processing()
mae,mse,sqr = forecasting1(df_item, df_emi, emission_list)

NameError: name 'pre_process_df' is not defined

In [ ]:
# Evaluation metrics
#mae,mse,sqr = forecasting(df_item, df_emi, emission_list)
average_mean_absolute_error = (sum(mae)/len(mae))*100
average_mean_squared_error = (sum(mse)/len(mse))*100
average_sqrt = (sum(sqr)/len(sqr))*100
print('Mean absolute error:', average_mean_absolute_error)
print('Mean squared error:', average_mean_squared_error)
print('Root mean squared error:', average_sqrt)